# Create Stunning Fractal Art with Python: A Tutorial For Beginners And Hardcore Math Lovers
## With a single line of code or even less


### Introduction

The phrase "I've never seen anything more beautiful" should only be used for fractals. Sure, there is the Mona Lisa, Van Gokh's paintings (which all have been ruined by AI-generated art, by the way), but I don't think any artist or human, could create anything as royally amazing as fractals. 

On the left, we have the iconic fractal, the Mandelbrot's set, discovered in NNNN when Python or no graphing software was available. 

![](images/opening.gif)

The Mandelbrot's set is a set of complex numbers that when plotted with a few tricks, forms the self-repeating shape we see. Every number in the set can also be a seed to __Julia sets__ and you can see the beauties appearing as I move around the mouse inside boundary of the Mandelbrot's set. 

But before we can plot the Mandelbrot or Julia sets (but, believe me, we will), we have got a quite ground to cover. If you were here just to see the cool pictures, I highly recommend the Fractal Art gallery or download the open-source Fraqtive software, which I used to generate the above GIF and the one below:

![](images/zoom.gif)

If you just want to plot the Mandelbrot set in Python with a single line of code, here it is (no, the subtitle was not clickbait):

Paste the code block here.

But if you want to go down the beautiful rabbit hole of fractals and learn how to plot them and most importantly, color them appropriately, then read on, possibly for a couple more articles. In this post, we will only learn how to plot a black-white vanilla Mandelbrot using Matplotlib and NumPy. Then, we will take things to a whole new level with Pillow in the future articles.

Let's get started. 